In [ ]:
import os
from pathlib import Path

notebook_dir = Path.cwd()
layout_dir = notebook_dir.parent.parent / "layout"

if Path.cwd().resolve() != layout_dir.resolve():
    os.chdir(layout_dir)
import gdsfactory as gf
from blocks import *
from comb_drive_tuning import *
import pandas as pd
import warnings
from dowhen import when
warnings.filterwarnings("ignore")

import sys
sys.path.insert(0, str(notebook_dir))


In [ ]:
def round_inner_corner(c:gf.Component, inner_radius) -> gf.Component:
    c_out = gf.Component()
    reg = c.get_region('WG',merge=True).rounded_corners(r_inner=inner_radius*1000, r_outer=0, n=16)
    c_out.add_polygon(reg, 'WG')
    c_out.add_ref(c.extract(layers=[l for l in c.layers if gf.get_layer(l) != gf.get_layer('WG')]))
    c_out.ports = c.ports
    return c_out
def extract_wg(c:gf.Component):
    c_out = gf.Component()
    polys = c.get_polygons(layers=["WG"], merge=True)[gf.get_layer("WG")]
    if not polys:
        return c_out

    for poly in polys:
        c_out.add_polygon(poly, layer=gf.get_layer("WG"))
    return c_out
def extract_period(c:gf.Component, finger_length, finger_width, finger_gap, overlap) -> gf.Component:
    c_out = gf.Component()
    box_height = finger_gap*2+2*finger_width
    box_width = finger_length*2 + overlap+10
    box = gf.kdb.Region(gf.kdb.Box(w=box_width*1000, h=box_height*1000))
    intersection = c.get_region('WG', merge=True)
    intersected = intersection & box
    c_out.add_polygon(intersected, layer=gf.get_layer("WG"))
    return c_out
finger_param = {
    "finger_length": 20,
    "finger_width": 3,
    'overlap': 1,
    'finger_gap': 2,
}
when(combdrive_array,"movable_base = perforated_shaft").do("movable_base = perforated_shaft(width=movable_base_width, height=base_length,brick_mode=1,hole_size=(20,20), margin=10,create_mask=False)").goto("movable_base_ref = c.add_ref(movable_base)")
finger_spec = partial(combdrive_fingers_5um,pair_num=1,**finger_param)
comb_array = combdrive_array(finger_spec=finger_spec, movable_base_width=30,fixed_base_width=30).copy()
comb_array_wg = extract_period(round_inner_corner(extract_wg(comb_array), inner_radius=2), **finger_param)
comb_array_wg.write_gds(notebook_dir/"comb_array_period.gds",with_metadata=False)
comb_array

In [ ]:
import mph
client = mph.start()

In [ ]:
model = client.load(notebook_dir / "comb.mph")
jmodel = model.java

In [ ]:
model.parameter('xmax',f'{comb_array_wg.xmax}[um]')
model.parameter('ymax',f'{comb_array_wg.ymax}[um]')

In [ ]:
from tqdm.notebook import tqdm
finger_param = {
    "finger_length": 20,
    "finger_width": 2,
    'overlap': 5.2,
    'finger_gap': 2,
}
def sim_combdrive(comb_array_wg):
    finger_spec = partial(combdrive_fingers_5um,pair_num=1,**finger_param)
    comb_array = combdrive_array(finger_spec=finger_spec, movable_base_width=30,fixed_base_width=30).copy()
    comb_array_wg = extract_period(round_inner_corner(extract_wg(comb_array), inner_radius=2), **finger_param)
    comb_array_wg.write_gds(notebook_dir/"comb_array_period.gds",with_metadata=False)
    model.parameter('xmax',f'{comb_array_wg.xmax}[um]')
    model.parameter('ymax',f'{comb_array_wg.ymax}[um]')
    imp1 = model/'geometries'/'Geometry 1'/'Import 1'
    imp1.java.importData()
    model.solve()
    results = model.evaluate(['-es.Forcex_FF'],['nN'])
    return results
df_results = sim_combdrive(comb_array_wg)
df_results

In [ ]:
from tqdm.notebook import tqdm

# Parameter sweep with resume/checkpoint

# sweep ranges
finger_lengths = [12]
overlaps = [round(0.1 * i, 3) for i in range(1, 200, 3)]  # 0.1 ... 5.0

checkpoint_path = notebook_dir / "sweep_results.csv"

# load checkpoint
if checkpoint_path.exists():
    df_sweep = pd.read_csv(checkpoint_path)
    done = set(zip(
        df_sweep.loc[df_sweep['Forcex_nN'].notna(), 'finger_length'],
        df_sweep.loc[df_sweep['Forcex_nN'].notna(), 'overlap']
    ))
else:
    df_sweep = pd.DataFrame(columns=['finger_length', 'overlap', 'Forcex_nN'])
    done = set()

for fl in tqdm(finger_lengths, desc="finger_length"):
    for ov in tqdm(overlaps, desc=f"overlap (fl={fl})", leave=False):
        # constraint: finger_length > overlap + 2
        if fl <= ov + 2:
            continue
        key = (fl, ov)
        if key in done:
            continue

        # update global finger_param used by sim_combdrive()
        finger_param.update({'finger_length': fl, 'overlap': ov})

        try:
            res = sim_combdrive(comb_array_wg)
            # normalize result
            force_val = None
            if isinstance(res, pd.DataFrame):
                if not res.empty:
                    force_val = float(res.iloc[0, 0])
            elif isinstance(res, dict):
                v = next(iter(res.values()))
                try:
                    force_val = float(v)
                except Exception:
                    force_val = None
            elif isinstance(res, (list, tuple)):
                try:
                    force_val = float(res[0])
                except Exception:
                    force_val = None
            else:
                try:
                    force_val = float(res)
                except Exception:
                    force_val = None

            df_sweep = pd.concat(
                [df_sweep, pd.DataFrame([{'finger_length': fl, 'overlap': ov, 'Forcex_nN': force_val}])],
                ignore_index=True
            )
            df_sweep.to_csv(checkpoint_path, index=False)
            done.add(key)
        except Exception:
            # record failed attempt; it will be retried on resume
            df_sweep = pd.concat(
                [df_sweep, pd.DataFrame([{'finger_length': fl, 'overlap': ov, 'Forcex_nN': None}])],
                ignore_index=True
            )
            df_sweep.to_csv(checkpoint_path, index=False)

df_sweep

In [ ]:
# Plot Forcex_nN vs overlap for each finger_length
df_plot = (
    df_sweep[df_sweep.finger_length==15].dropna(subset=['Forcex_nN'])
            .sort_values('overlap')
            .pivot(index='overlap', columns='finger_length', values='Forcex_nN')
)

ax = df_plot.plot(marker='o', linewidth=1.5)
ax.set_xlabel('Overlap (um)')
ax.set_ylabel('Force (nN)')
ax.set_title('Force vs overlap for different finger lengths')
ax.grid(True)
ax.set_ylim((2.1,2.5))
ax.set_xlim((0.1,10))
# save figure
# ax.figure.savefig(notebook_dir / 'force_vs_overlap.png', dpi=150, bbox_inches='tight')

In [ ]:
model.clear()
model.reset()
model.save()